In [72]:
import numpy as np
import pandas as pd
import sklearn
import sklearn.datasets
import sklearn.linear_model
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
import seaborn as sns

In [60]:
#Importing csv data
states_data = pd.read_csv('4.18states.csv')
counties = pd.read_csv('abridged_couties.csv')
confirmed = pd.read_csv('time_series_covid19_confirmed_US.csv')
deaths = pd.read_csv('time_series_covid19_deaths_US.csv')

In [92]:
#Cleaning Data to answer the following questions
#We will first create tables only where California is concerned
ca_data = states_data[states_data["Province_State"] == 'California']
ca_counties = counties[counties["State"] == 'California'][["countyFIPS", 'PopulationEstimate2018', "PopulationDensityperSqMile2010", "MedicareEnrollment,AgedTot2017", "SVIPercentile"]]
ca_deaths = deaths[deaths["Province_State"] == 'California']

#We only care about the total deaths, which is found in column 4/18/20
ca_deaths = ca_deaths[["FIPS", "4/18/20"]]
ca_deaths.columns = ["FIPS", "deaths"]

confirmed_ca = confirmed[confirmed["Province_State"] == "California"]
#We only care about total confirmed cases, which is also the column 4/18/20
confirmed_ca = confirmed_ca[["FIPS", "4/18/20"]]
confirmed_ca.columns = ["FIPS", "confirmed"]

#convert String FIPS to float so we can use it in conjunction with other tables
ca_counties["countyFIPS"] = ca_counties["countyFIPS"].apply(lambda x: float(x))
#Join deaths and confirmed columns into counties table using FIPS. Remove counties without sufficient death data.
ca_counties = ca_counties.join(other = ca_deaths, lsuffix = "countyFIPS", rsuffix = "FIPS")
ca_counties = ca_counties.join(other = confirmed_ca, lsuffix = "countyFIPS", rsuffix = "FIPS")
ca_counties = ca_counties.fillna(-1)
ca_counties = ca_counties[ca_counties["deaths"] >= 0]
ca_counties = ca_counties[ca_counties["confirmed"] >= 0]
ca_counties = ca_counties.drop(["FIPSFIPS", "FIPScountyFIPS"], axis = 1)
ca_counties

,countyFIPS,PopulationEstimate2018,PopulationDensityperSqMile2010,"MedicareEnrollment,AgedTot2017",SVIPercentile,deaths,confirmed
191,6015.0,27828.0,28.4,4740.0,0.9713,41.0,1114.0
192,6017.0,190678.0,106.0,36864.0,0.1838,0.0,2.0
193,6019.0,994400.0,156.2,115579.0,0.9611,0.0,7.0
194,6021.0,28047.0,21.4,4613.0,0.8895,0.0,16.0
195,6023.0,136373.0,37.7,22746.0,0.7605,0.0,12.0
196,6025.0,181827.0,41.8,25375.0,0.9978,0.0,3.0
197,6027.0,17987.0,1.8,4036.0,0.5701,19.0,685.0
198,6029.0,896764.0,103.3,91664.0,0.9659,0.0,2.0
199,6031.0,151366.0,110.1,13838.0,0.9707,0.0,36.0
200,6033.0,64382.0,51.5,13415.0,0.9178,7.0,311.0


In [93]:
#We conduct Train Test Split here, train data is 0.8 of our original and test data is 0.2
tr, te = train_test_split(ca_counties, test_size=0.2)

In [97]:
#Our training features don't require the IDs, the death count, or the confirmed count.
#Our used features are 2018 Population, Population Density, Medicare Enrollment, SVI Percentile (an indicator of at risk cities)
x_tr = tr.drop(['countyFIPS', 'deaths', 'confirmed'], axis = 1)
y_tr = tr["confirmed"]
x_te = te.drop(['countyFIPS', 'deaths', 'confirmed'], axis = 1)
y_te = te["confirmed"]

#Using Logistic Regression, we predict the number of confirmed cases 
model = LogisticRegression()
model.fit(x_tr, y_tr)
y_pred = model.predict(x_te)

def computeMSE(y, y_hat):
    return 1/(len(y)) * np.sum((y - y_hat)**2)

print(computeMSE(y_te, y_pred))


1492536.1875


Initially, we wished to train on the California data because we wanted to predict if different confirmed cases in Californian counties. However, because many Californian counties didn't actually report their number of cases, we lost a lot of data. Having too little training data has negatively impacted our performance, as can be seen by computing MSE. Next, we will attempt to use the entirety of the US as training data.

In [108]:
#This follows the same process as the first few cells. We only take the features we want and rename the date column to something more meaningful.
clean_counties = counties[["countyFIPS", 'PopulationEstimate2018', "PopulationDensityperSqMile2010", "MedicareEnrollment,AgedTot2017", "SVIPercentile"]]
clean_deaths = deaths[["FIPS", "4/18/20"]]
clean_deaths.columns = ["FIPS", "deaths"]
clean_confirm = confirmed[["FIPS", "4/18/20"]]
clean_confirm.columns = ["FIPS", "confirmed"]
#Hardcoding the removal of two 0-data rows, labeled City 1 and City 2
clean_counties = clean_counties[clean_counties["countyFIPS"] != 'City1']
clean_counties = clean_counties[clean_counties["countyFIPS"] != 'City2']
clean_counties["countyFIPS"] = clean_counties["countyFIPS"].apply(lambda x: float(x))

#Join deaths and confirmed columns into counties table using FIPS. Remove counties without sufficient death data.
clean_counties = clean_counties.join(other = clean_deaths, lsuffix = "countyFIPS", rsuffix = "FIPS")
clean_counties = clean_counties.join(other = clean_confirm, lsuffix = "countyFIPS", rsuffix = "FIPS")
clean_counties = clean_counties.fillna(-1)
clean_counties = clean_counties[clean_counties["deaths"] >= 0]
clean_counties = clean_counties[clean_counties["confirmed"] >= 0]
clean_counties = clean_counties.drop(["FIPSFIPS", "FIPScountyFIPS"], axis = 1)
#We're going to do a remove rows where deaths and confirmed are 0, because they haven't been infected so are not good indicators.
clean_counties = clean_counties[clean_counties["deaths"] + clean_counties["confirmed"] != 0]